In [52]:
#!pip install tabulate
#rapidfuzz psycopg2 matplotlib seaborn

In [56]:
import pandas as pd
import os
import logging
import numpy as np
from rapidfuzz import process, fuzz

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("clean_airbnb_data.log"), # Nuevo nombre de log
        logging.StreamHandler()
    ]
)

logging.info("Inicio de la extración de datos de AirBnB.")

try:
    SCRIPT_DIR = os.path.dirname(os.path.abspath(__file__))
except NameError:
    SCRIPT_DIR = os.getcwd()

BASE_DIR = os.path.dirname(os.path.dirname(SCRIPT_DIR))
RAW_DATA_DIR = os.path.join(BASE_DIR, 'data', 'raw')
CSV_FILE_NAME = 'Airbnb_Open_Data.csv'
CSV_FILE_PATH = os.path.join(RAW_DATA_DIR, CSV_FILE_NAME)

logging.info(f"Ruta del archivo CSV construida con os: {CSV_FILE_PATH}")

df_airbnb = pd.DataFrame()
logging.info("DataFrame 'df_airbnb' predefinido como un DataFrame vacío.")

try:
    logging.info(f"Intentando cargar el archivo CSV: {CSV_FILE_PATH}")
    if not os.path.exists(CSV_FILE_PATH):
        logging.error(f"Error: Archivo CSV no encontrado en '{CSV_FILE_PATH}'")
        raise FileNotFoundError(f"Archivo no encontrado: {CSV_FILE_PATH}")
    df_airbnb = pd.read_csv(CSV_FILE_PATH, low_memory=False)
    logging.info(f"Archivo CSV '{CSV_FILE_PATH}' cargado exitosamente.")
    logging.info(f"El DataFrame original tiene {df_airbnb.shape[0]} filas y {df_airbnb.shape[1]} columnas.")
except FileNotFoundError:
    raise
except Exception as e:
    logging.error(f"Ocurrió un error al cargar el CSV '{CSV_FILE_PATH}': {e}")
    raise

if not df_airbnb.empty:
    logging.info("Verificando filas duplicadas en df_airbnb.")
    num_duplicados = df_airbnb.duplicated().sum()
    logging.info(f"Número de filas duplicadas encontradas en df_airbnb: {num_duplicados}")
else:
    logging.critical("El DataFrame df_airbnb está vacío después de la carga. Terminando el script.")
    # Considerar salir del script si el DataFrame está vacío: exit()

# --- Limpieza Preliminar y Conversión de Tipos de Datos (Refactorizado) ---
logging.info("Iniciando limpieza preliminar y conversión de tipos de datos (versión optimizada).")
df_cleaned = pd.DataFrame()

if not df_airbnb.empty:
    df_cleaned = df_airbnb.copy()
    logging.info("Copia de df_airbnb creada como df_cleaned.")

    original_columns = df_cleaned.columns.tolist()
    df_cleaned.columns = df_cleaned.columns.str.lower().str.replace(' ', '_', regex=False).str.replace('[^0-9a-zA-Z_]', '', regex=True)
    new_columns = df_cleaned.columns.tolist()
    logging.info(f"Columnas de df_cleaned normalizadas.")
    if original_columns != new_columns:
        logging.info(f"Cambios en nombres de columnas: {dict(zip(original_columns, new_columns))}")
    else:
        logging.info("Nombres de columnas ya estaban normalizados o no requirieron cambios significativos.")

    def clean_string_column(series, col_name):
        logging.debug(f"Limpiando columna string: {col_name}")
        series = series.astype(str).str.strip().replace({'nan': pd.NA, '': pd.NA, 'None': pd.NA})
        return series

    def to_numeric_column(series, col_name, numeric_type='Int64'):
        logging.debug(f"Convirtiendo columna a numérica ({numeric_type}): {col_name}")
        nulls_before = series.isna().sum()
        if numeric_type == 'datetime':
            series = pd.to_datetime(series, format='%m/%d/%Y', errors='coerce')
        else:
            series = pd.to_numeric(series, errors='coerce')
            if numeric_type == 'Int64' and not series.empty:
                if series.dropna().apply(lambda x: x.is_integer()).all() or series.dropna().empty:
                    series = series.astype('Int64')
                else:
                    logging.warning(f"Columna '{col_name}' contiene flotantes, no se convertirá a Int64, se mantendrá como float.")
        
        coerced_nulls = series.isna().sum() - nulls_before
        if coerced_nulls > 0:
            logging.warning(f"Columna '{col_name}': {coerced_nulls} nuevos NaNs/NaTs por coerción.")
        return series

    def standardize_categorical_fuzz(series, col_name, choices_list, score_cutoff=85):
        logging.debug(f"Estandarizando columna categórica con RapidFuzz: {col_name}")
        unique_values = series.dropna().unique()
        mapping = {}
        for val in unique_values:
            match = process.extractOne(str(val), choices_list, scorer=fuzz.WRatio, score_cutoff=score_cutoff)
            if match:
                mapping[val] = match[0]
            else:
                mapping[val] = val
        
        original_na_mask = series.isna()
        series_mapped = series.map(mapping)
        series_mapped[original_na_mask] = pd.NA
        
        changes = (series.dropna() != series_mapped.dropna()).sum()
        if changes > 0:
            logging.info(f"Columna '{col_name}': {changes} valores estandarizados usando RapidFuzz.")
        return series_mapped

    try:
        numeric_cols_int = ['construction_year', 'minimum_nights', 'number_of_reviews', 'review_rate_number', 
                            'calculated_host_listings_count', 'availability_365']
        
        for col in numeric_cols_int:
            if col in df_cleaned.columns:
                df_cleaned[col] = to_numeric_column(df_cleaned[col], col, 'Int64')
            else: logging.warning(f"Columna '{col}' no encontrada para conversión numérica (Int64).")

        numeric_cols_float = ['lat', 'long', 'reviews_per_month']
        for col in numeric_cols_float:
            if col in df_cleaned.columns:
                df_cleaned[col] = to_numeric_column(df_cleaned[col], col, 'float')
            else: logging.warning(f"Columna '{col}' no encontrada para conversión numérica (float).")

        df_cleaned = df_cleaned.reset_index(drop=True)
        df_cleaned['id'] = df_cleaned.index + 1
        df_cleaned = df_cleaned.reset_index(drop=True)
        df_cleaned['host_id'] = df_cleaned.index + 150000
        
        string_cols = ['name', 'host_name']
        for col in string_cols:
            if col in df_cleaned.columns:
                df_cleaned[col] = clean_string_column(df_cleaned[col], col)
            else: logging.warning(f"Columna '{col}' no encontrada para limpieza de string.")

        categorical_cols_pre_fuzz = ['neighbourhood_group', 'neighbourhood']
        for col in categorical_cols_pre_fuzz:
            if col in df_cleaned.columns:
                df_cleaned[col] = clean_string_column(df_cleaned[col], col)
                if col == 'neighbourhood_group' and col in df_cleaned.columns:
                    canonical_groups = ['Manhattan', 'Brooklyn', 'Queens', 'Bronx', 'Staten Island']
                    if not df_cleaned[col].dropna().empty:
                       df_cleaned[col] = standardize_categorical_fuzz(df_cleaned[col], col, canonical_groups, score_cutoff=80)
                       logging.info(f"RapidFuzz aplicado a '{col}'.")
                    else:
                       logging.info(f"Columna '{col}' está vacía o solo nulos, RapidFuzz no aplicado.")

                nunique_threshold = 50 if col == 'neighbourhood' else 20 
                if col in df_cleaned.columns and df_cleaned[col].nunique(dropna=False) < nunique_threshold:
                    df_cleaned[col] = df_cleaned[col].astype('category')
                    logging.info(f"Columna '{col}' convertida a category.")
                elif col in df_cleaned.columns:
                    logging.info(f"Columna '{col}' limpiada (no convertida a category debido a alta cardinalidad: {df_cleaned[col].nunique(dropna=False)}).")

            else: logging.warning(f"Columna '{col}' no encontrada para limpieza categórica.")

        category_cols_direct = ['cancellation_policy', 'room_type']
        for col in category_cols_direct:
            if col in df_cleaned.columns:
                df_cleaned[col] = clean_string_column(df_cleaned[col], col).astype('category')
                logging.info(f"Columna '{col}' convertida a category.")
            else: logging.warning(f"Columna '{col}' no encontrada para conversión a category.")
        
        # Fechas
        if 'last_review' in df_cleaned.columns:
            df_cleaned['last_review'] = to_numeric_column(df_cleaned['last_review'], 'last_review', 'datetime')
            logging.info("Columna 'last_review' convertida a datetime.")
        else: logging.warning("Columna 'last_review' no encontrada.")

        # Booleanas (con mapeo)
        if 'host_identity_verified' in df_cleaned.columns:
            verified_map = {'verified': True, 'unconfirmed': False}
            df_cleaned['host_verification'] = df_cleaned['host_identity_verified'].map(verified_map).astype('boolean')
            logging.info("Columnas 'host_verification', creada a partir de 'host_identity_verified'.")
        else: logging.warning("Columna 'host_identity_verified' no encontrada.")

        if 'instant_bookable' in df_cleaned.columns:
            bookable_map = {'TRUE': True, 'FALSE': False, 'True': True, 'False': False, 'true': True, 'false': False} # Cubrir varias capitalizaciones
            df_cleaned['instant_bookable_flag'] = df_cleaned['instant_bookable'].astype(str).str.upper().map(bookable_map).astype('boolean')
            logging.info("Columna 'instant_bookable_flag' creada a partir de 'instant_bookable'.")
        else: logging.warning("Columna 'instant_bookable' no encontrada.")

        currency_cols = {'price': 'price', 'service_fee': 'service_fee'}
        for original_col, new_col_numeric in currency_cols.items():
            if original_col in df_cleaned.columns:
                series_cleaned_str = df_cleaned[original_col].astype(str).str.replace('$', '', regex=False).str.replace(',', '', regex=False).str.strip().replace({'nan': pd.NA, '': pd.NA})
                df_cleaned[new_col_numeric] = to_numeric_column(series_cleaned_str, new_col_numeric, 'float')
                logging.info(f"Columna '{new_col_numeric}' creada a partir de '{original_col}'.")
            else: logging.warning(f"Columna original '{original_col}' no encontrada para procesar moneda.")

        cols_to_drop_original_names = ['host_identity_verified', 'instant_bookable', 'country', 'country_code', 'license', 'house_rules']
        
        normalized_cols_to_drop = []
        for col_name in cols_to_drop_original_names:
            normalized_name = col_name.lower().replace(' ', '_').replace('[^0-9a-zA-Z_]', '')
            if normalized_name in df_cleaned.columns:
                normalized_cols_to_drop.append(normalized_name)
            elif col_name in df_cleaned.columns:
                normalized_cols_to_drop.append(col_name)

        existing_cols_to_drop = [col for col in normalized_cols_to_drop if col in df_cleaned.columns]
        if existing_cols_to_drop:
            df_cleaned.drop(columns=existing_cols_to_drop, inplace=True, errors='ignore')
            logging.info(f"Columnas {existing_cols_to_drop} eliminadas de df_cleaned.")
        
        logging.info("Proceso de limpieza preliminar y conversión de tipos optimizado completado.")

    except KeyError as ke:
        logging.error(f"Ocurrió un KeyError durante la limpieza: '{ke}'. Verifica que la columna exista en df_cleaned (posiblemente después de la normalización).")
        logging.error(f"Columnas disponibles en df_cleaned: {df_cleaned.columns.tolist()}")
        logging.error(f"Ocurrió un KeyError: '{ke}'. Revisa los nombres de las columnas y la lógica de normalización.")
    except Exception as e:
        logging.error(f"Ocurrió un error general durante la limpieza: {e}", exc_info=True)
        logging.error(f"Ocurrió un error general: {e}")

else:
    logging.warning("El DataFrame df_airbnb está vacío. No se puede realizar la limpieza.")

logging.info("Proceso finalizado exitosamente.")

2025-05-17 21:37:24,152 - INFO - Inicio de la extración de datos de AirBnB.
2025-05-17 21:37:24,154 - INFO - Ruta del archivo CSV construida con os: \\wsl.localhost\Ubuntu\home\y4xul\Proyecto_ETL\data\raw\Airbnb_Open_Data.csv
2025-05-17 21:37:24,164 - INFO - DataFrame 'df_airbnb' predefinido como un DataFrame vacío.
2025-05-17 21:37:24,167 - INFO - Intentando cargar el archivo CSV: \\wsl.localhost\Ubuntu\home\y4xul\Proyecto_ETL\data\raw\Airbnb_Open_Data.csv
2025-05-17 21:37:25,153 - INFO - Archivo CSV '\\wsl.localhost\Ubuntu\home\y4xul\Proyecto_ETL\data\raw\Airbnb_Open_Data.csv' cargado exitosamente.
2025-05-17 21:37:25,155 - INFO - El DataFrame original tiene 102599 filas y 26 columnas.
2025-05-17 21:37:25,157 - INFO - Verificando filas duplicadas en df_airbnb.
2025-05-17 21:37:25,300 - INFO - Número de filas duplicadas encontradas en df_airbnb: 541
2025-05-17 21:37:25,302 - INFO - Iniciando limpieza preliminar y conversión de tipos de datos (versión optimizada).
2025-05-17 21:37:25,3